In [1]:
!pip install pandas 
!pip install numpy
#!pip install matplotlib
#!pip install datetime
#!pip install seaborn

import pandas as pd 
import numpy as np
#import matplotlib.pyplot as plt
#import datetime as dt
#import seaborn as sns

# Создание вспомогательных функций

Создадим функции, которые упростят процесс вывода всей необходимой информации о датасете.

In [2]:
# Функция для обзора датасета
def overview_df(df):
    print(" HEAD OF DATAFRAME ".center(100,'-'), '\n', df.head())
    print(" INFO OF DATAFRAME ".center(100,'-'))
    df.info()
    print(" SHAPE ".center(100,'-'), '\n', 'Rows: {}'.format(df.shape[0]), '\n', 'Columns: {}'.format(df.shape[1]))
    print(" TYPES ".center(100,'-'), '\n', df.dtypes)
    print(" NUMBER OF UNIQUES ".center(100,'-'), '\n', df.nunique())
    print(" MISSING VALUES ".center(100,'-'), '\n', missing_values(df))
    print(" DUPLICATED VALUES ".center(100,'-'), '\n', cheking_duplicates(df))
    print(" DESCRIPTIVE STATISTICS ".center(100,'-'), '\n', df.describe().T)

# Функция для поиска пропущенных значений
def missing_values(df):
    mn = df.isnull().sum().sort_values(ascending = False)
    mp = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    missing_values = pd.concat([mn, mp], axis = 1, keys = ['Missing_Number', 'Missing_Percent'])
    if mn.sum() > 0:
        return missing_values[missing_values['Missing_Number'] > 0]
    else:
        return 'There are no missing values.'

# Функция для проверки и подсчета количества дубликатов
def cheking_duplicates(df):
    duplicate_values = df.duplicated(keep='first').sum()
    if duplicate_values > 0:
        df.drop_duplicates(keep='first', inplace=True)
        return 'The number of duplicated values is ' + str(duplicate_values) + '. Duplicates were dropped. New shape is ' + str(df.shape)
    else:
        return 'There are no duplicate values.'

# Первичный обзор датасета

С помощью написанной функции overview_df(df) выведем всю информацию о датасете.

In [3]:
df0 = pd.read_csv(r'C:\Users\Александра\Desktop\customer.csv')
df = df0.copy()

overview_df(df)

---------------------------------------- HEAD OF DATAFRAME ----------------------------------------- 
            InvoiceDate InvoiceNo StockCode  \
0  2010-12-01 08:26:00    536365    85123A   
1  2010-12-01 08:26:00    536365     71053   
2  2010-12-01 08:26:00    536365    84406B   
3  2010-12-01 08:26:00    536365    84029G   
4  2010-12-01 08:26:00    536365    84029E   

                           Description  Quantity  UnitPrice  CustomerID  Age  \
0   WHITE HANGING HEART T-LIGHT HOLDER         6       2.55     17850.0   77   
1                  WHITE METAL LANTERN         6       3.39     17850.0   77   
2       CREAM CUPID HEARTS COAT HANGER         8       2.75     17850.0   77   
3  KNITTED UNION FLAG HOT WATER BOTTLE         6       3.39     17850.0   77   
4       RED WOOLLY HOTTIE WHITE HEART.         6       3.39     17850.0   77   

   Gender        Income         Country  
0  female  26300.794657  United Kingdom  
1  female  26300.794657  United Kingdom  
2  female  26

# Basic data cleaning

Дубликаты уже были посчитаны и удалены во время составления первичного обзора датасета. 
Далее перейдем к описанию данных, содержащихся в каждом из столбцов, и разберемся, какие из них и почему необходимо почистить.
Почередно проверим, все ли данные из столбцов подходят для анализа, и сделаем выводы о полученной информации.

## Столбец 0 InvoiceDate

In [4]:
# Первая покупка в датасете
print('The minimum date is:', df.InvoiceDate.min())

# Последняя покупка в датасете
print('The maximum date is:', df.InvoiceDate.max())

The minimum date is: 2010-12-01 08:26:00
The maximum date is: 2011-12-09 12:50:00


Из собранных данных можно заметить, что первая запись в датасете относится к 1 декабря 2010 года в 8:26, а последняя — к 9 декабря 2011 года в 12:50.
Исходя из этого, можем исключить декабрь 2011 года из анализа, поскольку нет полной информации об этом месяце.

In [5]:
# Удаление записи из датафрейма за декабрь 2011 года
df = df[df.InvoiceDate < '2011-12-01']

## Столбец 1 InvoiceNo

In [6]:
# Номера операций/заказов (InvoiceNo), начинающиеся с символа 'C', который указывает на возвращенные или отмененные заказы
df[df['InvoiceNo'].str.startswith('C') == True]

,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
141,2010-12-01 09:41:00,C536379,D,Discount,-1,27.50,14527.0,35,female,30268.760531,United Kingdom
154,2010-12-01 09:49:00,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,4.65,15311.0,38,female,33513.444454,United Kingdom
235,2010-12-01 10:24:00,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,1.65,17548.0,17,male,20649.029315,United Kingdom
236,2010-12-01 10:24:00,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,0.29,17548.0,17,male,20649.029315,United Kingdom
237,2010-12-01 10:24:00,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,0.29,17548.0,17,male,20649.029315,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...
516379,2011-11-30 17:39:00,C579886,22197,POPCORN HOLDER,-1,0.85,15676.0,50,male,42712.949947,United Kingdom
516380,2011-11-30 17:39:00,C579886,23146,TRIPLE HOOK ANTIQUE IVORY ROSE,-1,3.29,15676.0,50,male,42712.949947,United Kingdom
516381,2011-11-30 17:42:00,C579887,84946,ANTIQUE SILVER T-LIGHT GLASS,-1,1.25,16717.0,27,female,31161.429914,United Kingdom
516382,2011-11-30 17:42:00,C579887,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,-1,7.95,16717.0,27,female,31161.429914,United Kingdom


In [7]:
# Удаление всех номеров операций (InvoiceNo), начинающихся с символа 'C'
df = df[df['InvoiceNo'].str.startswith('C') != True]

In [8]:
# Проверка количества уникальных номеров операций (InvoiceNo)
df.InvoiceNo.nunique()

21195

## Столбец 2 StockCode

In [9]:
# Проверка количества уникальных биржевых кодов-идентификаторов (StockCode) среди всех записей о продажах
df.StockCode.nunique()

4056

In [10]:
# Топ-10 биржевых кодов-идентификаторов (StockCode), которые продавались чаще всего
df.StockCode.value_counts().head(10)

StockCode
85123A    2192
85099B    2058
22423     1951
47566     1672
20725     1538
84879     1430
22720     1354
22197     1350
21212     1315
22383     1274
Name: count, dtype: int64

## Столбец 3 Description

In [11]:
# Проверка количества уникальных описаний товаров (Description) среди всех записей о продажах
df.Description.nunique()

4198

In [12]:
# Топ-10 проданных "описаний" товаров (Description)
df.Description.value_counts().head(10)

Description
WHITE HANGING HEART T-LIGHT HOLDER    2253
JUMBO BAG RED RETROSPOT               2058
REGENCY CAKESTAND 3 TIER              1948
PARTY BUNTING                         1672
LUNCH BAG RED RETROSPOT               1537
ASSORTED COLOUR BIRD ORNAMENT         1429
SET OF 3 CAKE TINS PANTRY DESIGN      1350
PACK OF 72 RETROSPOT CAKE CASES       1315
LUNCH BAG  BLACK SKULL.               1254
JUMBO BAG PINK POLKADOT               1210
Name: count, dtype: int64

Обратим внимание, что бывают записи, в которых описания товаров (Description) содержат некоторую информацию, не относящуюся к продажам. 

In [13]:
# Проверка данных, где записи описаний (Description) состоят из '?' или начинаются с '?'
df[df['Description'].str.startswith('?') == True]

,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
7313,2010-12-03 16:50:00,537032,21275,?,-30,0.0,NaN,25,female,32130.475292,United Kingdom
21518,2010-12-09 14:48:00,538090,20956,?,-723,0.0,NaN,25,female,32130.475292,United Kingdom
38261,2010-12-20 10:36:00,539494,21479,?,752,0.0,NaN,25,female,32130.475292,United Kingdom
43662,2011-01-04 16:53:00,540100,22837,?,-106,0.0,NaN,25,female,32130.475292,United Kingdom
50806,2011-01-10 10:04:00,540558,21258,?,-29,0.0,NaN,25,female,32130.475292,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...
472842,2011-11-16 13:11:00,576765,22219,???lost,-550,0.0,NaN,25,female,32130.475292,United Kingdom
493686,2011-11-23 12:37:00,578245,22568,?? missing,-170,0.0,NaN,25,female,32130.475292,United Kingdom
497819,2011-11-24 12:45:00,578476,72807B,????missing,-124,0.0,NaN,25,female,32130.475292,United Kingdom
497820,2011-11-24 12:45:00,578477,72807A,???missing,-224,0.0,NaN,25,female,32130.475292,United Kingdom


In [14]:
# Заметим, что при этом CustomerId = NaN и цена за единицу товара UnitPrice = 0
# Удаление всех полученных записей
df = df[df['Description'].str.startswith('?') != True]

In [15]:
# Проверяем данных, где записи описаний (Description) начинаются с '*' 
df[df['Description'].str.startswith('*') == True]

,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
20749,2010-12-09 14:09:00,538071,21120,*Boombox Ipod Classic,1,16.98,NaN,25,female,32130.475292,United Kingdom
35675,2010-12-17 14:54:00,539437,20954,*USB Office Mirror Ball,1,8.47,NaN,25,female,32130.475292,United Kingdom
37095,2010-12-17 17:08:00,539453,20954,*USB Office Mirror Ball,1,8.47,NaN,25,female,32130.475292,United Kingdom


In [16]:
# Заметим, что при этом CustomerId = NaN
# Изменение записей на соответствующие описания (Description) без символа '*' и перевод в верхний регистр, как и во всех остальных записях
df['Description'] = df['Description'].replace(('*Boombox Ipod Classic','*USB Office Mirror Ball'),
                                             ('BOOMBOX IPOD CLASSIC','USB OFFICE MIRROR BALL'))

In [17]:
# В описаниях товаров (Description) есть записи строчными (в нижнем регистре) буквами - это "шумы" в датасете, которые можно удалить
df[df['Description'].str.islower() == True]['Description'].value_counts()

Description
check                                  146
damaged                                 43
damages                                 42
found                                   24
sold as set on dotcom                   20
                                      ... 
wrong code                               1
wrong code?                              1
did  a credit  and did not tick ret      1
sold as 22467                            1
lost??                                   1
Name: count, Length: 113, dtype: int64

In [18]:
# Удаление всех найденных неподходящих записей-"шумов"
df = df[df['Description'].str.islower() != True]

In [19]:
# В описаниях товаров (Description) есть записи, 
# которые начинаются с заглавной буквы и не соответствуют типу большинства записей - это "шумы" в датасете, которые можно удалить
df[df['Description'].str.istitle() == True]['Description'].value_counts()

Description
Manual                   315
Next Day Carriage         69
Damaged                   14
Bank Charges              11
Found                      8
Amazon                     4
High Resolution Image      3
Adjustment                 2
Dagamed                    1
Damages                    1
Breakages                  1
Amazon Adjustment          1
Show Samples               1
Crushed                    1
Missing                    1
Display                    1
Dotcom                     1
John Lewis                 1
Name: count, dtype: int64

In [20]:
# Удаление всех найденных неподходящих записей-"шумов"
df = df[df['Description'].str.istitle() != True]

In [21]:
# Преобразование записей описаний (Description) к единому виду, удалив лишние пробелы с начала и конца строки
df['Description'] = df['Description'].str.strip()

## Столбец 4 Quantity

In [22]:
# Посмотрим на описательные статистики количества (Quantity) в первичном обзоре датасета
# Заметим, что существуют отрицательные значения, которые могут указывать на отмененные/возвращенные заказы

# Оставим только те записи, у которых положительные значения количества (Quantity), что по нашему предположению говорит о совершенных заказах
df = df[df['Quantity'] >= 0]

## Столбец 5 UnitPrice

In [23]:
# Посмотрим на распределение значений цены за единицу товара (UnitPrice) по перцентилям
df.UnitPrice.describe(percentiles = [0.25, 0.5, 0.75, 0.85,0.95, 0.99])

count    500654.000000
mean          3.716555
std          38.644772
min      -11062.060000
25%           1.250000
50%           2.080000
75%           4.130000
85%           5.790000
95%           9.950000
99%          16.980000
max       13541.330000
Name: UnitPrice, dtype: float64

In [24]:
# Удаление записей с нулевой ценой за единицу товара (UnitPrice = 0)
df = df[df['UnitPrice'] != 0.0]

## Столбец 6 CustomerID

In [25]:
# Проверка количество уникальных идентификаторов клиентов (CustomerID)
df.CustomerID.nunique()

4293

In [26]:
# Проверка записей, в которых идентификатор клиента (CustomerID) имеет пустые значения
df[df.CustomerID.isnull()]

,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
1443,2010-12-01 14:32:00,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2.51,NaN,25,female,32130.475292,United Kingdom
1444,2010-12-01 14:32:00,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2.51,NaN,25,female,32130.475292,United Kingdom
1445,2010-12-01 14:32:00,536544,21786,POLKADOT RAIN HAT,4,0.85,NaN,25,female,32130.475292,United Kingdom
1446,2010-12-01 14:32:00,536544,21787,RAIN PONCHO RETROSPOT,2,1.66,NaN,25,female,32130.475292,United Kingdom
1447,2010-12-01 14:32:00,536544,21790,VINTAGE SNAP CARDS,9,1.66,NaN,25,female,32130.475292,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...
515863,2011-11-30 15:33:00,579787,21888,BINGO SET,2,7.46,NaN,25,female,32130.475292,United Kingdom
515864,2011-11-30 15:33:00,579787,21912,VINTAGE SNAKES & LADDERS,1,7.46,NaN,25,female,32130.475292,United Kingdom
515865,2011-11-30 15:33:00,579787,21916,SET 12 RETRO WHITE CHALK STICKS,1,0.79,NaN,25,female,32130.475292,United Kingdom
515866,2011-11-30 15:33:00,579787,21929,JUMBO BAG PINK VINTAGE PAISLEY,1,4.13,NaN,25,female,32130.475292,United Kingdom


In [27]:
# Удаление записей, в которых идентификатор клиента (CustomerID) имеет пустые значения
df = df[~df.CustomerID.isnull()]

## Столбец 7 Age

In [28]:
# Проверка количества уникальных совпадений по возрастам клиентов (Age) среди всех записей
df.Age.value_counts(normalize = True)

Age
28    0.043529
44    0.041155
33    0.040454
29    0.038387
42    0.035887
        ...   
76    0.000099
85    0.000085
89    0.000083
86    0.000059
97    0.000037
Name: proportion, Length: 80, dtype: float64

In [29]:
# Проверка количества уникальных совпадений по возрастам клиентов (Age) среди всех записей, 
# сгруппированных по 10 отрезкам возрастов (Age) по убыванию
df.Age.value_counts(normalize = True, bins = 10)

(23.5, 32.0]      0.267180
(32.0, 40.5]      0.223081
(40.5, 49.0]      0.202607
(14.914, 23.5]    0.116434
(49.0, 57.5]      0.089336
(57.5, 66.0]      0.074434
(66.0, 74.5]      0.019024
(74.5, 83.0]      0.002883
(83.0, 91.5]      0.002806
(91.5, 100.0]     0.002214
Name: proportion, dtype: float64

In [30]:
# Топ-10 возрастов клиентов (Age)
df.Age.value_counts().head(10)

Age
28    16337
44    15446
33    15183
29    14407
42    13469
26    12567
35    12296
27    11717
45    10904
32    10830
Name: count, dtype: int64

## Столбец 8 Gender

In [31]:
# Проверка количества уникальных совпадений по полу клиентов (Gender) среди всех записей
df.Gender.value_counts(normalize = True)

Gender
female    0.649569
male      0.350431
Name: proportion, dtype: float64

## Столбец 9 Income

In [32]:
# Проверка количества уникальных совпадений по доходу клиентов (Income) среди всех записей
df.Income.value_counts(normalize = True)

Income
32904.146809    0.019195
32584.481215    0.014252
42889.070502    0.012209
31798.776856    0.010940
31798.349432    0.006866
                  ...   
33368.292910    0.000003
34402.492658    0.000003
42802.803797    0.000003
33923.638207    0.000003
31410.720362    0.000003
Name: proportion, Length: 4293, dtype: float64

In [33]:
# Проверка количества уникальных совпадений по доходу клиентов (Income) среди всех записей, 
# сгруппированных по 10 отрезкам доходов клиентов (Income) по убыванию
df.Income.value_counts(normalize = True, bins = 10)

(32293.966, 35020.214]    0.445275
(29567.718, 32293.966]    0.154687
(40472.71, 43198.958]     0.151192
(18635.462, 21388.974]    0.122570
(43198.958, 45925.206]    0.057619
(37746.462, 40472.71]     0.033716
(35020.214, 37746.462]    0.012637
(26841.47, 29567.718]     0.011457
(24115.222, 26841.47]     0.007159
(21388.974, 24115.222]    0.003688
Name: proportion, dtype: float64

## Столбец 10 Country

In [34]:
# Проверка количества уникальных совпадений по стране (Country) среди всех записей
df.Country.value_counts(normalize = True)

Country
United Kingdom          0.888229
Germany                 0.023178
France                  0.021308
EIRE                    0.018398
Spain                   0.006432
Netherlands             0.006056
Belgium                 0.005161
Switzerland             0.004905
Portugal                0.003584
Australia               0.003147
Norway                  0.002651
Italy                   0.001998
Channel Islands         0.001974
Finland                 0.001788
Cyprus                  0.001604
Sweden                  0.001194
Austria                 0.001042
Denmark                 0.000983
Poland                  0.000879
Japan                   0.000855
Israel                  0.000653
Unspecified             0.000642
Singapore               0.000573
Iceland                 0.000456
Canada                  0.000402
USA                     0.000392
Malta                   0.000298
Greece                  0.000290
United Arab Emirates    0.000181
European Community      0.000160
RS

In [35]:
# Заметим, около 90% записей о стране (Country) относятся к Великобратании (United Kingdom)
# Разделим все записи о странах (Country) на две категории: "United Kingdom" и "Others", которая будет объединять все остальные страны
df['Country'] = df['Country'].apply(lambda x: 'United Kingdom' if x == 'United Kingdom' else 'Others')

In [36]:
# Заново выполним проверку количества уникальных совпадений по стране (Country) среди всех записей
df.Country.value_counts(normalize = True)

Country
United Kingdom    0.888229
Others            0.111771
Name: proportion, dtype: float64

# Вторичный обзор датасета после базовой чистки данных


In [37]:
# Вывод обзора датасета с помощью той же написанной функции
overview_df(df)

---------------------------------------- HEAD OF DATAFRAME ----------------------------------------- 
            InvoiceDate InvoiceNo StockCode  \
0  2010-12-01 08:26:00    536365    85123A   
1  2010-12-01 08:26:00    536365     71053   
2  2010-12-01 08:26:00    536365    84406B   
3  2010-12-01 08:26:00    536365    84029G   
4  2010-12-01 08:26:00    536365    84029E   

                           Description  Quantity  UnitPrice  CustomerID  Age  \
0   WHITE HANGING HEART T-LIGHT HOLDER         6       2.55     17850.0   77   
1                  WHITE METAL LANTERN         6       3.39     17850.0   77   
2       CREAM CUPID HEARTS COAT HANGER         8       2.75     17850.0   77   
3  KNITTED UNION FLAG HOT WATER BOTTLE         6       3.39     17850.0   77   
4       RED WOOLLY HOTTIE WHITE HEART.         6       3.39     17850.0   77   

   Gender        Income         Country  
0  female  26300.794657  United Kingdom  
1  female  26300.794657  United Kingdom  
2  female  26

# Исследовательский анализ данных (Exploratory Data Analysis - EDA) 


# Когортный анализ


# RFM-анализ
